In [20]:
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from chromadb import PersistentClient
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import sqlite3

In [ ]:
conn = sqlite3.connect(r'C:\Users\SRINU\Downloads\eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

In [ ]:
# Reading the columns of Table
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

In [ ]:
# Loading the Database Table inside a Pandas DataFrame
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

In [ ]:
# Applying the above Function on the Entire Data
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

In [ ]:
import random

random_df = df.sample(frac=0.3, random_state=42)

print(random_df.shape)

random_df.head()

In [ ]:
random_df.to_csv("random_sampled_data.csv",index=False)

In [ ]:
import os
os.getcwd()

In [ ]:
df1=pd.read_csv(r"random_sampled_data.csv")

In [ ]:
print(df1.shape)
df1.head()

In [12]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SRINU\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SRINU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SRINU\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def clean_subtitle(subtitle):
    # Remove timestamps
    clean_content = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n?', '', subtitle)

    # Remove other non-textual patterns
    clean_content = re.sub(r'<[^>]+>', '', clean_content)

    clean_content = re.sub(r"[^\w\s]", '', clean_content)

    clean_content = re.sub(r"[^\x00-\x7F]+", '', clean_content)

    clean_content = re.sub(r"\b\d+\s", '', clean_content)

    # Convert to lowercase
    clean_content = clean_content.lower()

    # Tokenize the subtitle content
    tokens = word_tokenize(clean_content)

    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words('english'))

    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words]

    # Join the filtered tokens back into a string
    clean_content = ' '.join(clean_tokens)

    return clean_content.strip()

In [ ]:
df1['cleaned_content'] = df1['file_content'].apply(clean_subtitle)

In [ ]:
df1.head()

### **4. (Must Implement) A very important step to improve the performance: Document Chunker.**

a. Consider the challenge of embedding large documents: Information Loss.

b. It is often not practical to embed an entire document as a single vector, particularly when dealing with long documents.

c. Solution: Divide a large document into smaller, more manageable chunks for embedding.

d. Another Problem: Let’s say we set the token window to be 500, then we’d expect each chunk to be just below 500 tokens. One common concern of this method is that we might accidentally cut off some important text between chunks, splitting up the context. To mitigate this, we can set overlapping windows with a specified amount of tokens to overlap so we have tokens shared between chunks.

In [ ]:
df1.to_csv("cleaned_data.csv",index=False)

In [ ]:
df=pd.read_csv("cleaned_data.csv")

In [ ]:
df.shape

In [11]:
import nltk
from nltk.tokenize import word_tokenize

# Function to count tokens
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

# Iterate through data and count tokens for each file
for index, entry in df.iterrows():
    total_tokens = count_tokens(entry["cleaned_content"])
    print(f"File: {entry['name']}, Number of Tokens: {total_tokens}")

NameError: name 'df' is not defined

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

In [ ]:
# Define the document chunker function
def document_chunker(document, chunk_size=1500, overlap_size=200):
    tokens = word_tokenize(document)  # Tokenize the document
    num_tokens = len(tokens)
    chunks = []

    # Iterate over the tokens and create chunks with specified overlap
    for start in range(0, num_tokens, chunk_size - overlap_size):
        end = min(start + chunk_size, num_tokens)
        chunk = tokens[start:end]
        chunks.append({'name': row['name'], 'chunk_index': len(chunks) + 1, 'chunk_text': ' '.join(chunk)})

    return chunks

In [ ]:
# Define the document chunker function
def document_chunker(document, chunk_size=1500, overlap_size=200):
    tokens = word_tokenize(document)  # Tokenize the document
    num_tokens = len(tokens)
    chunks = []

    # Iterate over the tokens and create chunks with specified overlap
    for start in range(0, num_tokens, chunk_size - overlap_size):
        end = min(start + chunk_size, num_tokens)
        chunk = tokens[start:end]
        chunks.append({'name': row['name'], 'chunk_index': len(chunks) + 1, 'chunk_text': ' '.join(chunk)})

    return chunks

In [ ]:
# Apply the document chunker to each cleaned document in the DataFrame
all_chunks = []
for index, row in df.iterrows():
    document = row['cleaned_content']
    document_chunks = document_chunker(document)
    all_chunks.extend(document_chunks)

# Create a new DataFrame from the list of chunks
chunks_df = pd.DataFrame(all_chunks)

In [ ]:
print(chunks_df.shape)

chunks_df.head()

In [ ]:
chunks_df.to_csv("chunks_data.csv",index=False)

### **3. Experiment with the following to generate text vectors of subtitle documents:**

a. BOW/TFIDFto generate sparse vector representations. Note that this will only help you to build a Keyword Based Search Engine.

b. BERT based “SentenceTransformers” to generate embeddings which encode
 semantic information. This can help us build a Semantic Search Engine.

In [3]:
df1 = pd.read_csv(r"chunks_data.csv")

print(df1.shape)

df1.head()

(62637, 3)


,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [4]:
df1['chunk_text'][111]

'advertise product brand contact wwwopensubtitlesorg today congratulation rabbi oh well rabbi wife asks tell im lasri send father warm gratitude chandelier beautiful im asked right educational method yeshiva headmaster turned complained yeshiva student went ski vacation ski vacation five day complained ignore keep theyre good men study excel wanted day break expel yeshiva unacceptable every time student finish mishna chapter theyll go trip school would stay empty gmara book would left unread corner torah headmaster asked correct educational method teach boy way impose mountain basin answer ill tell right method right method daniel paran dori medium darset present created written directed eliran malka creator main editor daniel paran directed maor zaguri new black gedaliah stop bugging sorry rabbi come need come well yesterday awesome im gedaliah hi im talus usually break ice begin date story first date rabbi ovadia wife told information im going study torah wont clean wont cook wont an

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Calculate TF-IDF for the documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df1['chunk_text'])

In [13]:

cleaned_query = clean_subtitle(query)

# Calculate TF-IDF for the query
query_tfidf = tfidf_vectorizer.transform([cleaned_query])

# Calculate cosine similarity between the query and documents
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

# Get the indices of documents sorted by similarity score
sorted_indices = cosine_similarities.argsort()[0][::-1]

In [14]:

# Return top 5 most similar chunk_text
top_k = 5
top_documents = df1.loc[sorted_indices[:top_k], 'chunk_text'].tolist()
print("Top 5 most similar chunk_text:")
for i, text in enumerate(top_documents, 1):
    print(f"Document {i}: {text}")

Top 5 most similar chunk_text:
Document 1: advertise product brand contact wwwopensubtitlesorg today congratulation rabbi oh well rabbi wife asks tell im lasri send father warm gratitude chandelier beautiful im asked right educational method yeshiva headmaster turned complained yeshiva student went ski vacation ski vacation five day complained ignore keep theyre good men study excel wanted day break expel yeshiva unacceptable every time student finish mishna chapter theyll go trip school would stay empty gmara book would left unread corner torah headmaster asked correct educational method teach boy way impose mountain basin answer ill tell right method right method daniel paran dori medium darset present created written directed eliran malka creator main editor daniel paran directed maor zaguri new black gedaliah stop bugging sorry rabbi come need come well yesterday awesome im gedaliah hi im talus usually break ice begin date story first date rabbi ovadia wife told information im goin

In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [16]:
df1=pd.read_csv("chunks_data.csv")
print(df1.shape)
df1.head()

(62637, 3)


,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [17]:
# Encode the chunk text to get semantic embeddings
chunk_embeddings = model.encode(df1['chunk_text'].tolist())

# Add the embeddings to the DataFrame
df1['embeddings'] = chunk_embeddings.tolist()

In [18]:
df1.shape

(62637, 4)

### **5. Store embeddings in a ChromaDB database.**

In [22]:
import chromadb

In [23]:
client=chromadb.Client()

In [24]:
collection = client.create_collection(
        name="searchengine",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [25]:
for i, embedding in enumerate(chunk_embeddings):
    collection.add(ids=f"chunk_{i}",  # Replace with a more descriptive ID scheme if needed
                   embeddings=embedding.tolist(),  # Convert to a list for Chroma
                   documents=df1.loc[i, "chunk_text"],  # Use chunk_text as documents
                   metadatas={"name": df1.loc[i, "name"]}  # Use 'name' column data as metadata
    )


# **Part 2: Retrieving Documents**

**1. Take the user's search query.**

**2. Preprocess the query (if required).**

**3. Create query embedding.**

**4. Using cosine distance, calculate the similarity score between embeddings of documents and user search query embedding.**

**5. These cosine similarity scores will help in returning the most relevant candidate documents as per user's search query.**

In [26]:
from sentence_transformers import SentenceTransformer, util

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [27]:
subtitle_text = "help kiss excuse im gon na kiss need facial facial ouch thats gon na hurt tell straight english dont want wear wont force try okay go ahead dont like doesnt unlike doesnt like unli unlike fine give like doesnt dont right stay still sir tep really hurt love miss monica remember know say love hurt wow looking good hmm think look good mmm modern cool whats last one posh posh posh mama mae yeah ill posh mmhmm let posh mama mae ow ow im sorry hurt done bit ow ooh ow hand hurt sorry sorry sir please hold still look yuck dinner fork good salad fork good mmm okay "
cleaned_subtitle = clean_subtitle(subtitle_text)

In [28]:

# Encode the cleaned subtitle text to get semantic embeddings
query_embedding = model.encode([cleaned_subtitle])

In [29]:
# Perform a query on the ChromaDB collection using the embeddings
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
    include=['documents']
)


In [30]:
documents = results['documents']

# Iterate over the documents and print each document
for i, query_documents in enumerate(documents):
    for j, document in enumerate(query_documents):
        print(f"Document {i+1}, Item {j+1}: {document}")



Document 1, Item 1: teeth last time wed done twice would really embarrassed idve thought oh god one u obviously bad kisser harry oh god im shocked mum youre one million card call boy say special talent pulling milfs feel like annamay deep wanted kiss kai cant lie teddy bear chain saying sergeant snake boy told girl joining army stop seeing rude olivias one bit long like roundup kiss hold one like ouch shaq ever find lied well shes gon na see see weve got hair removal cream boy blistered entire body using hairremoval cream lad holiday right kiss frog find prince fake foot boy claim nan initial tattooed foot really girl slept lad holiday uh oh trouble somethings come along burst bubble ive got menu say porn cocktail chicken tikka massageher dessert ice makeherscream oh god thats vibe wow boy caught sex restaurant waiter oh feel like ive kissed boy honestly think kissed challenge girl cleaned well actually theyve cleaned girl back villa shaqs realising come tanya he got major feeling coin